# WRF QGPV perturbation inversion calculation

In [1]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import netCDF4
from netCDF4 import Dataset
from wrf import tk, to_np, getvar, CoordPair, destagger, smooth2d, interplevel, vinterp, ll_to_xy, latlon_coords,get_cartopy,ALL_TIMES
import time
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import metpy.calc as mpcalc
from metpy.units import units

from wrf import (cartopy_xlim, cartopy_ylim, extract_times, interpz3d)

In [2]:
#Define type of tile and desired pressure level
#fullphys_expdom or nolatentheat_expdom
filetype = "fullphys_expdom"                                   #CHANGE

In [3]:
wrfout_fname = 'wrfout_'+filetype+'.nc'  # trajectory output
wrfout = Dataset(wrfout_fname, 'r')  # Dataset is the class behavior to open the file

In [4]:
msfm = wrfout.variables['MAPFAC_M'][0, :, :]     # Map scale factor on mass grid
cor = wrfout.variables['F'][0,:, :]     # Coriolis parameter
ds = 1./wrfout.variables['RDX'][0]     # grid spacing SAME IN ZONAL AND MERIDIONAL DIRECTIONS
# Get the latitude and longitude points# Get the sea level pressure
slp = getvar(wrfout, "slp", timeidx=ALL_TIMES)
lats, lons = latlon_coords(slp)
# Get the cartopy mapping object
cart_proj = get_cartopy(slp)

In [5]:
#Define constants
R = 287.05 #J kg^-1 K^-1
kappa=0.2856219 #Rd / cp
cp = 1004.7 #J kg^-1 K^-1
pref = 1000. #hPa
g = 9.80665 #gravity
omega = 7.292e-5 #Earth's rotation rate (s^-1)
a = 2.e7 / np.pi #Radius of the earth (m)
p0=1.e5
alpha=-1./5.255877
gamma = 0.0065 #lapse rate, (K/m)
f0 = np.average(cor)
dp = 50.e2

In [6]:
from standard_atmosphere import standard_atmosphere
pb = 1000.
pb_half = 975.
pt = 100.
pt_half = 125.
p_levels = np.linspace(pb,pt,19)
p_half_levels = np.linspace(pb_half,pt_half,18)

ZS,phiS,TS,SS=standard_atmosphere(p_levels, p_half_levels)

In [7]:
#Get times from dataset
times = extract_times(wrfout, None)
count = np.count_nonzero(times)
#Set t, k, i, j indice lengths
tlen = len(times)
klen = len(p_levels)
jlen = np.shape(msfm)[0]
ilen = np.shape(msfm)[1]

In [8]:
#Format times to be in standard format
dattimes = []
import pandas as pd
for i in range (0, count):
    dattimes.append(str(pd.Timestamp(times[i])))

In [9]:
# Geopotential height
z = getvar(wrfout, "z", units="m", timeidx=ALL_TIMES)
T = getvar(wrfout,"tk", timeidx=ALL_TIMES)

z_full = np.zeros((tlen,klen,jlen,ilen))
T_full = np.zeros((tlen,klen,jlen,ilen))

for t in np.arange(tlen):
    z_full[t,:,:,:] = vinterp(wrfout, z[t,:,:,:], 'pressure', p_levels, extrapolate=True, field_type='z', log_p=True, timeidx=t,meta=False)
    T_full[t,:,:,:] = vinterp(wrfout, T[t,:,:,:], 'pressure', p_levels, extrapolate=True, field_type='tk', log_p=True, timeidx=t,meta=False)

In [10]:
def geostrophic_vorticity(tlen,num_levs,num_lats,num_lons,msfm,ds,f0,phi_prime):
    import numpy as np

    g=9.8066
    qgpv_geovor = np.zeros([tlen,num_levs,num_lats,num_lons])

    AC_1 = np.zeros([num_lats,num_lons])
    AC_2 = np.zeros([num_lats,num_lons])
    AC_3v = np.zeros([num_lats,num_lons])
    AC_4 = np.zeros([num_lats,num_lons])
    AC_5 = np.zeros([num_lats,num_lons])

    for j in np.arange(1,num_lats-1):
        for i in np.arange(1,num_lons-1): 
            AC_1[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_2[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_4[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_5[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_3v[j,i] = (-4*msfm[j,i]**2.)/(f0*ds*ds)

    for t in np.arange(tlen):
        for j in np.arange(1,num_lats-1):
            for i in np.arange(1,num_lons-1):
                qgpv_geovor[t,:,j,i]=AC_1[j,i]*phi_prime[t,:,j-1,i]+AC_2[j,i]*phi_prime[t,:,j,i-1]+ \
                AC_3v[j,i]*phi_prime[t,:,j,i]+AC_4[j,i]*phi_prime[t,:,j,i+1]+AC_5[j,i]*phi_prime[t,:,j+1,i]

    return qgpv_geovor

In [11]:
num_lats=lats.shape[0]
num_lons=lons.shape[1]
num_levs=p_levels.size

g=9.8066

f0 = np.average(cor)
phi_prime=np.zeros([tlen,p_levels.size,num_lats,num_lons])

for t in np.arange(tlen):
    for k in np.arange(1,p_levels.size):
        phi_prime[t,k,:]=g*(z_full[t,k,:]-ZS[k])
    
qgpv_geovor=geostrophic_vorticity(tlen,num_levs,num_lats,num_lons,msfm,ds,f0,phi_prime)

In [12]:
def stretching_term(num_levs,num_lats,num_lons,msfm,ds,dp,f0,S,phip):

    AC_3s=np.zeros([num_levs])
    AC_6=np.zeros([num_levs])
    AC_7=np.zeros([num_levs]) 
    
    for k in np.arange(1,18):
        AC_6[k]=f0/(S[k-1]*dp**2.)
        AC_7[k]=f0/(S[k]  *dp**2.)
        AC_3s[k]= -(AC_6[k]+AC_7[k])
    strat=np.zeros([tlen,num_levs,num_lats,num_lons])
    
    for k in np.arange(1,num_levs-1):
        for j in np.arange(1,num_lats-1):
            for i in np.arange(1,num_lons-1):
                strat[t,k,j,i] = AC_3s[k]*phip[t,k,j,i]+ AC_6[k]*phip[t,k-1,j,i]+ AC_7[k]*phip[t,k+1,j,i]
    return strat

In [13]:
qgpv_strtch = stretching_term(num_levs,num_lats,num_lons,msfm,ds,dp,f0,SS,phi_prime)
qgpv = qgpv_geovor + qgpv_strtch

In [14]:
qgpv_mean = np.zeros((klen,jlen,ilen))
for k in np.arange(klen):
    for j in np.arange(jlen):
        for i in np.arange(ilen):
            qgpv_mean[k,j,i]= np.squeeze(np.mean(qgpv[0:,k,j,i])) #Taking time mean of QGPV

In [15]:
qgpvp = np.zeros([tlen,num_levs,num_lats,num_lons])
for t in np.arange(tlen):
    for k in np.arange(klen):
        qgpvp[t,k,:] = qgpv[t,k,:]-qgpv_mean[k,:]

In [16]:
# Geopotential height
hght_all = getvar(wrfout, "z", units="m", timeidx=ALL_TIMES)

z_all = vinterp(wrfout, hght_all, 'pressure', p_levels, extrapolate=True, field_type='z', log_p=True, timeidx=ALL_TIMES,meta=False)

In [17]:
zper = np.zeros((tlen,klen,jlen,ilen))
for t in np.arange(tlen):
    for k in np.arange(klen):
        zper[t,k,:,:] = (z_all[t,k,:,:] - ZS[k])

In [18]:
zper_mean = np.zeros((klen,jlen,ilen))
for k in np.arange(klen):
    for j in np.arange(jlen):
        for i in np.arange(ilen):
            zper_mean[k,j,i]= np.squeeze(np.mean(zper[0:,k,j,i]))

In [19]:
%%time
z_prime = np.zeros((tlen,klen,jlen,ilen))
for t in np.arange(tlen):
    for k in np.arange(klen):
        z_prime[t,k,:,:] = zper[t,k,:,:] - zper_mean[k,:,:]

CPU times: user 68.7 ms, sys: 81.5 ms, total: 150 ms
Wall time: 149 ms


In [20]:
dz_b = (z_prime[:,1,:,:]-z_prime[:,0,:,:])
dz_t = (z_prime[:,-1,:,:]-z_prime[:,-2,:,:])

Tb = ((g*dz_b)/R)/np.log(p_levels[0]/p_levels[1]) #pert temp at bottom level
Tt = ((g*dz_t)/R)/np.log(p_levels[-2]/p_levels[-1]) #pert temp at top level

In [21]:
def qgpv_invert(iterations,tlen,num_levs,num_lats,num_lons,p_levels,f0, ds, dp,msfm, SS,qgpv, Tb, Tt, BC, BC_flag):   
    
    import numpy as np
    
    R = 287.04
    g = 9.8066
    
#BC_flag = 0 for Dirichlet and 1 for Neumann
    res=np.zeros([tlen,num_levs,num_lats,num_lons])
    phip = BC
    

# define coefficients

    AC_3s=np.zeros([num_levs])
    AC_6=np.zeros([num_levs])
    AC_7=np.zeros([num_levs]) 
    
    for k in np.arange(17):
        AC_6[k+1]=f0/(SS[k]*dp**2.)
        AC_7[k+1]=f0/(SS[k+1]  *dp**2.)
        AC_3s[k+1]= -(AC_6[k+1]+AC_7[k+1])

    mfs=np.zeros([num_lats,num_lons])
    mfs[:] = (f0*ds*ds)/(msfm[:]**2.)

    AI_1 = 1.
    AI_2 = 1.
    AI_4 = 1.
    AI_5 = 1.
    AI_3v = -4.

    AI_3s=np.zeros([num_levs,num_lats,num_lons])
    AI_6=np.zeros([num_levs,num_lats,num_lons])
    AI_7=np.zeros([num_levs,num_lats,num_lons])

    for k in np.arange(num_levs):
        AI_6[k,:]  = mfs[:]*AC_6[k]
        AI_7[k,:]  = mfs[:]*AC_7[k] 
        AI_3s[k,:] = mfs[:]*AC_3s[k]

#####
    for t in np.arange(tlen):
        for num_iter in np.arange(iterations):
            for k in np.arange(1,num_levs-1):
                for j in np.arange(1,num_lats-1):
                    for i in np.arange(1,num_lons-1):
                        RES = AI_1*phip[t,k,j-1,i] + \
                        AI_2*phip[t,k,j,i-1] + \
                        (AI_3v+AI_3s[k,j,i])*phip[t,k,j,i] + \
                        AI_4*phip[t,k,j,i+1] + \
                        AI_5*phip[t,k,j+1,i] + \
                        AI_6[k,j,i]*phip[t,k-1,j,i] + \
                        AI_7[k,j,i]*phip[t,k+1,j,i] - qgpv[t,k,j,i]*mfs[j,i]
                
                        phip[t,k,j,i] = phip[t,k,j,i] - 1.8*RES/(AI_3v+AI_3s[k,j,i])
                        res[t,k,j,i]=RES
            if(BC_flag==1):
                if(k==1): 
                    phip[t,0,:] = phip[t,1,:] - (R * Tb[t,:]/(100*p_levels[0]))*dp
                if(k==num_levs-2): 
                    phip[t,-1,:] = phip[t,-2,:] + (R * Tt[t,:]/(100.*p_levels[-1]))*dp
        if(np.amax(res/g)<.25): 
            print('stopping at iteration number: ', num_iter)
            break
    
    return phip,res,num_iter

In [22]:
%%time
iterations=50
#BC effectivley sets first guess for inversion (here it is set to zero everywhere)
BC =np.zeros([tlen,p_levels.size,num_lats,num_lons])
res=np.zeros([tlen,p_levels.size,num_lats,num_lons])

#qgpv_anomaly = np.zeros([p_levels.size,num_lats,num_lons])
#for k in range(p_levels.size):
#    BC[k,:]=9.81*(z_full[k,:]-ZS[k])
BC_flag=1
Tb=np.zeros([tlen,num_lats,num_lons])
Tt=np.zeros([tlen,num_lats,num_lons])


phip=np.zeros([p_levels.size,num_lats,num_lons])
phip,res,num_iter=qgpv_invert(iterations,tlen,num_levs,num_lats,num_lons,p_levels,f0, ds, dp,msfm,SS,qgpvp, Tb, Tt, BC, BC_flag)

CPU times: user 1h 34min 49s, sys: 50.3 s, total: 1h 35min 39s
Wall time: 1h 33min 15s


In [23]:
print("Number of iterations: ", num_iter+1)
print("Average residual: ", np.mean(res))  
print("Absolute maximum residual: ", np.amax(res))

Number of iterations:  50
Average residual:  3.8369788840784424e-16
Absolute maximum residual:  62.44673272009962


In [24]:
#filetype = 'fullphys_expdom'

In [25]:
#Save data to file for use in other notebooks
import pickle
with open('inv_qgpv_full_'+filetype+'.pkl', 'wb') as f:
    pickle.dump([phip], f)

In [26]:
#Load data from saved file
#import pickle
#with open('inv_qgpv_tot_trop_'+filetype+'.pkl', 'rb') as f:
    #phip = pickle.load(f)
    
#phip = np.squeeze(phip)

In [27]:
#print(np.shape(phip))

In [28]:
lats, lons = latlon_coords(z)
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

In [29]:
lats = np.asarray(lats)
lons = np.asarray(lons)

In [30]:
lats_redo = lats
lons_redo = lons

In [31]:
#Account for cartopy plotting issue when longitude is less than 0.0
for j in range(jlen):
    for i in range(ilen):
        if lons_redo[j,i] < 0.0:
            lons_redo[j,i] = lons_redo[j,i] + 360.
        else:
            lons_redo[j,i] = lons_redo[j,i]

In [32]:
lats = lats_redo
lons = lons_redo

In [41]:
levtitle = str(input('Enter desired level for inversion (300, 500, 850, or 950): '))

Enter desired level for inversion (300, 500, 850, or 950):  1000


In [42]:
if levtitle == "300":
    lev = 14
elif levtitle == "500":
    lev = 10
elif levtitle == "850":
    lev = 3
elif levtitle == "900":
    lev = 2
elif levtitle == "950":
    lev = 1
elif levtitle == "1000":
    lev = 0

In [43]:
#calculate winds determined by qgpv perturbation from inversion
#U = np.zeros([tlen,klen,num_lats,num_lons])
#V = np.zeros([tlen,klen,num_lats,num_lons])

#for t in np.arange(tlen):
    #for k in np.arange(klen):
        #for j in np.arange(1,num_lats-1):
            #for i in np.arange(1,num_lons-1):
                #U[t,k,j,i] = -msfm[j,i]*0.5*(phip[t,k,j+1,i]-phip[t,k,j-1,i])/(f0*ds)
                #V[t,k,j,i] =  msfm[j,i]*0.5*(phip[t,k,j,i+1]-phip[t,k,j,i-1])/(f0*ds)

In [44]:
from scipy.ndimage.filters import gaussian_filter

In [45]:
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                     plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)

In [48]:
#PLOTTING FOR GEOPOTENTIAL HEIGHT ANOMALY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = phip[i,lev,:,:]
    tmp2 = slp[i,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-400,-380,-360,-340,-320,-300,-280,-260,-240,-220,-200,-180,-160,-140,
              -120,-100,-80,-60,-40,-20,20,40,60,80,100,120,140,
              160,180,200,220,240,260,280,300]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True,inline_spacing=-2.)

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Inverted Geopotential Height Anomaly (m^2 / s^2) from full QGPV Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/geopanom/geopanom_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

Working on time:  2019-11-25 12:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-25 15:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-25 18:00:00


...

/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-27 21:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-28 00:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


In [47]:
#PLOTTING FOR HEIGHT ANOMALY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = phip[i,lev,:,:] / g
    tmp2 = slp[i,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-40,-36,-32,-28,-24,-20,-16,-12,-8,-4,4,8,12,16,20,24,28,32,36,40]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="black")
    plt.clabel(qgcts, fmt='%.1f', inline=True, inline_spacing=-2.)

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Inverted Height Anomaly (m) from full QGPV Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/hghtanom/hghtanom_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

Working on time:  2019-11-25 12:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-25 15:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-25 18:00:00


...

/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-27 21:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Working on time:  2019-11-28 00:00:00


/opt/anaconda3/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


In [40]:
    #tmp2 = U[i,lev,:,:]
    #tmp3 = V[i,lev,:,:]
    
    #SCALE = .25
    #quiver_slices = (slice(None, None, 7), slice(None, None, 7))
    #quiver_kwargs = {'headlength': 4, 'headwidth': 3,
                 #'scale': 100}

    #ax.quiver(to_np(lons[quiver_slices]),to_np(lats[quiver_slices]), \
        #tmp2[::7,::7]*SCALE, tmp3[::7,::7]*SCALE,color='blue',\
        #transform=crs.PlateCarree(),**quiver_kwargs)